In [54]:
import sys
sys.path.append('/mnt/data/workspace/trip-overview/src/')
print(sys.path)

['/home/valentin.rudloff/source/trip-overview/prototype', '/home/valentin.rudloff/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles', '/home/valentin.rudloff/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/valentin.rudloff/source/trip-overview/venv/lib/python3.6/site-packages', '/home/valentin.rudloff/source/trip-overview/venv/lib/python3.6/site-packages/IPython/extensions', '/home/valentin.rudloff/.ipython', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/', '/mnt/data/workspace/trip-overview/src/']


In [55]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

def dist_from_gps(coord_a, coord_b):
    """
    Haversine’s algorithm
    param: decimal gps coord_a [lat, lon]
    param: decimal gps coord_b [lat, lon]
    result: distance between two gps coords
    """
    R = 6373.0 # approximate radius of earth in km

    lat1 = radians(coord_a[0])
    lon1 = radians(coord_a[1])
    lat2 = radians(coord_b[0])
    lon2 = radians(coord_b[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

gps_trace = pd.read_csv("gps.csv", header=None, names=["latitude", "longitude", "none"])[["latitude", "longitude"]]
gps_trace["altitude"] = 0.0
gps_trace["speed"] = 0.0
gps_trace["km"] = 0.0
for i in range(1, len(gps_trace["longitude"])):
    gps_trace["km"].iloc[i] = gps_trace["km"].iloc[i-1] + dist_from_gps(
        [gps_trace["latitude"].iloc[i-1], gps_trace["longitude"].iloc[i-1]],
        [gps_trace["latitude"].iloc[i], gps_trace["longitude"].iloc[i]])
gps_trace["current_country"] = "France"
gps_trace["current_step"] = 0
gps_trace['timestamp'] = range(1, 1 + len(gps_trace))
gps_trace

,latitude,longitude,altitude,speed,km,current_country,current_step,timestamp
0,48.87736,2.27966,0.0,0.0,0.000000,France,0,1
1,48.87736,2.27966,0.0,0.0,0.000000,France,0,2
2,48.87721,2.27951,0.0,0.0,0.019969,France,0,3
3,48.87720,2.27953,0.0,0.0,0.021807,France,0,4
4,48.87715,2.27967,0.0,0.0,0.033461,France,0,5
...,...,...,...,...,...,...,...,...
2720,48.54520,3.67221,0.0,0.0,129.176174,France,0,2721
2721,48.54501,3.67182,0.0,0.0,129.211831,France,0,2722
2722,48.54466,3.67063,0.0,0.0,129.307718,France,0,2723
2723,48.54455,3.67000,0.0,0.0,129.355697,France,0,2724


In [144]:
import folium
import folium.plugins
import base64
import branca
import datetime

# Custom Icons https://fontawesome.com/v5.15/icons?d=gallery&p=2


def get_road_trip_gps_trace(trace):
    # Change timestamp to datetime
    trace['date'] = pd.to_datetime(trace['timestamp'], unit='s')
    trace.reset_index(drop=True, inplace=True)
    # Interpolate the correct columns
    trace[["latitude", "longitude", "altitude", "speed", "km"]] = trace[["latitude", "longitude", "altitude", "speed", "km"]].interpolate(
        method='linear')
    # Fill forward current_country and current_step
    trace["current_country"].fillna(method='ffill', inplace=True)
    trace["current_step"].fillna(method='ffill', inplace=True)
    # Reset the current_step type
    trace["current_step"] = trace["current_step"].astype('int64')
    # Set current_step as index
    trace = trace.set_index(['current_step'])
    return trace

def get_sleeping_locations(data, static_position_threshold=1, min_distance=10):
    for current_step_trace_idx in range(gps_trace_clean.index.values[-1] + 1):
        current_step_trace = gps_trace_clean.loc[current_step_trace_idx]

gps_trace_clean = get_road_trip_gps_trace(gps_trace)
sleeping = get_sleeping_locations(gps_trace)

center_of_map = gps_trace_clean[["latitude", "longitude"]].iloc[-1].tolist() # Last updated GPS position
whole_trip_trace = gps_trace_clean[["latitude", "longitude"]]

# Maps
online_map = folium.Map(center_of_map, tiles="OpenStreetMap", attr="Capsule map")

# Markers group
sleep_position_group = folium.FeatureGroup(name="Campements")

# TODO Handle case where the sub step trace length is not > 1

for current_step_trace_idx in range(gps_trace_clean.index.values[-1] + 1):
    current_step_trace = gps_trace_clean.loc[current_step_trace_idx]
    if len(current_step_trace) > 1:
        distance_traveled_in_step = current_step_trace["km"].iloc[-1] - current_step_trace["km"].iloc[0]
        date = current_step_trace["date"].iloc[0].day
        """gif = "test.gif" # TODO
        tooltip_html = '<h1>{date}</h1><p>Etape {step}</p><p>Distance parcourue {distance} km</p><img src={gif}>'\
            .format(date=distance_traveled_in_step,
                    step=current_step_trace_idx,
                    distance=distance_traveled_in_step,
                    gif=gif)"""
        folium.plugins.AntPath(
            locations=current_step_trace[["latitude", "longitude"]],
            dash_array=[10, 15],
            delay=800,
            weight=6,
            color="#F6FFF3",
            pulse_color="#000000",
            paused=False,
            reverse=False,
            #tooltip=tooltip_html TODO
        ).add_to(online_map)
    
    current_sleep_location = current_step_trace[["latitude", "longitude"]].iloc[-1].values
    kw = {"prefix": "fa", "color": "blue", "icon": "bed"}
    html = '<h1>{date}</h1><p>Etape {step}</p><p>Distance parcourue {km} km</p><p>Coordonnée GPS: {lat}, {lon}</p>'.format(
        date=datetime.datetime.strftime(current_step_trace["date"].iloc[-1], "%d %B %Y"),
        step=current_step_trace_idx+1,
        km=round(current_step_trace["km"].iloc[-1], 1),
        lat=current_sleep_location[0], 
        lon=current_sleep_location[1])
    icon = folium.Icon(**kw)
    folium.Marker(list(current_sleep_location),
    icon=icon, tooltip=html).add_to(sleep_position_group)

#folium.LayerControl(collapsed=False).add_to(online_map)
sleep_position_group.add_to(online_map)
folium.LayerControl().add_to(online_map)
folium.plugins.LocateControl().add_to(online_map)

folium.plugins.Fullscreen(
    title="Agrandir",
    title_cancel="Annuler",
    force_separate_button=True,
).add_to(online_map)

    
legend_html = """
{{% macro html(this, kwargs) %}}
<!doctype html>
<html lang="fr">
<body>
<div id='maplegend' class='maplegend'
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
    border-radius:6px; padding: 10px;
    bottom: 10px;
    left: 10px;
    width: 180px;
    height: 120px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    opacity: 0.7;'>
    
<div class='legend-title'>Trip overview</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li>{travel_day} jours depuis le départ</li>
    <li>{km} km parcouru</li>
    <li>{country_crossed} pays visités</li>
    <li>mise à jour le {last_update}</li>
  </ul>
</div>
</div>

<div id='desc' class='maplegend'
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
    border-radius:6px; padding: 10px;
    bottom: 10px;
    left: 200px;
    width: 500px;
    height: 80px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    opacity: 0.7;'>
    
<div class='legend-title'>Description</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li>Voici une vue à jour du parcours de la capsule sur la carte du monde ! </li>
    <li>Voyageurs: Laurence et Emmanuel Rudloff</li>
  </ul>
</div>
</div>
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {{
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }}
  .maplegend .legend-scale ul {{
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }}
  .maplegend .legend-scale ul li {{
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }}
  .maplegend ul.legend-labels li span {{
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }}
  .maplegend .legend-source {{
    font-size: 80%;
    color: #777;
    clear: both;
    }}
  .maplegend a {{
    color: #777;
    }}
</style>
{{% endmacro %}}""".format(
  travel_day= (gps_trace["date"].iloc[-1]-gps_trace["date"].iloc[0]).days, 
  km=round(gps_trace["km"].iloc[-1], 1), 
  country_crossed= len(gps_trace["current_country"].value_counts()), 
  last_update="10 jun")
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
online_map.get_root().add_child(legend)

from folium.plugins import FloatImage
FloatImage(image="capsule.png", bottom=40, left=65).add_to(online_map)

# Limit bounds
online_map.fit_bounds(online_map.get_bounds())
online_map

In [ ]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Big</li>
    <li><span style='background:orange;opacity:0.7;'></span>Medium</li>
    <li><span style='background:green;opacity:0.7;'></span>Small</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

NameError: name 'arr' is not defined

In [3]:
l = [1, 4, 3, 1]
l[1:]

[4, 3, 1]